In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import multiprocessing
import time 
import collections
from datetime import datetime


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score,
    mean_absolute_error,
    confusion_matrix,
    classification_report
)

from tensorflow.python.platform import build_info as tf_build_info
from collections import Counter
print(tf_build_info.build_info)

from sklearn.preprocessing import StandardScaler
from tensorflow.keras import models, layers, regularizers, optimizers, callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model

OrderedDict([('cpu_compiler', 'C:/Program Files (x86)/Microsoft Visual Studio/2019/Community/VC/Tools/MSVC/14.29.30133/bin/HostX64/x64/cl.exe'), ('cuda_compute_capabilities', ['sm_35', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']), ('cuda_version', '64_112'), ('cudart_dll_name', 'cudart64_112.dll'), ('cudnn_dll_name', 'cudnn64_8.dll'), ('cudnn_version', '64_8'), ('is_cuda_build', True), ('is_rocm_build', False), ('is_tensorrt_build', False), ('msvcp_dll_names', 'msvcp140.dll,msvcp140_1.dll'), ('nvcuda_dll_name', 'nvcuda.dll')])


In [6]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_combined_model(model, X, data, n_samples=10):
    """
    Evaluates a combined model that outputs:
      - material_mean
      - material_std
      - build_price
      - setup_price
    """
    # --- 1. Run predictions ---
    y_pred = model.predict(X, verbose=0)
    material_mean = y_pred[0].flatten()
    material_std  = y_pred[1].flatten()
    build_pred    = y_pred[2].flatten()
    setup_pred    = y_pred[3].flatten()

    # --- 2. Actual values ---
    y_true_material = data["material_cost_per_truss"].values
    y_true_build    = data["build_cost_per_truss"].values
    y_true_setup    = data["setup_cost_per_run"].values

    # --- 3. Metrics ---
    mae_material = mean_absolute_error(y_true_material, material_mean)
    mae_build = mean_absolute_error(y_true_build, build_pred)
    mae_setup = mean_absolute_error(y_true_setup, setup_pred)

    print("\n🔍 Sample Predictions vs Actuals:\n")
    idxs = np.random.choice(len(X), size=min(n_samples, len(X)), replace=False)
    for i in idxs:
        print(f"Row {i}:")
        print(f"  Material → Pred: {material_mean[i]:6.2f} ± {material_std[i]:4.2f} | Actual: {y_true_material[i]:6.2f}")
        print(f"  Build    → Pred: {build_pred[i]:6.2f} | Actual: {y_true_build[i]:6.2f}")
        print(f"  Setup    → Pred: {setup_pred[i]:6.2f} | Actual: {y_true_setup[i]:6.2f}")
        print("-" * 60)

    print("\n📊 Summary Metrics:")

    return {
        "MAE_material": mae_material,
        "MAE_build": mae_build,
        "MAE_setup": mae_setup,
    }


In [7]:
data = pd.read_csv("C:/models/data/truss_rebalanced.csv", quotechar='"')

# Split features / target
X = data.drop(["material_cost_per_truss", "build_cost_per_truss", "setup_cost_per_run",], axis=1)
y = data["material_cost_per_truss"]


# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train: {len(X_train)} | Val: {len(X_val)}")


Train: 247600 | Val: 61900


In [8]:
material_model = tf.keras.models.load_model("C:/models/dev_model_scaled")
build_model    = tf.keras.models.load_model("C:/models/build_cost_price_model")
setup_model    = tf.keras.models.load_model("C:/models/setup_cost_price_model")

print("✅ All base models loaded successfully.")

✅ All base models loaded successfully.


In [9]:
# ---------------------------------------------------------
# 2️⃣ Define shared normalization + input layer
# ---------------------------------------------------------
# If your material model already includes a Normalization layer, skip this
# Otherwise, adapt normalization on your original raw X_train
normalizer = tf.keras.layers.Normalization()
normalizer.adapt(X_train)  # <-- unscaled raw data used during training

inputs = Input(shape=(X_train.shape[1],), name="truss_input")
norm_inputs = normalizer(inputs)

# ---------------------------------------------------------
# 3️⃣ Material cost predictions (ensemble mean + std)
# ---------------------------------------------------------
# NOTE: if your material model already outputs [mean, std],
# you can just call it directly:
material_mean, material_std = material_model(norm_inputs)


build_class = build_model(norm_inputs)
setup_class = setup_model(norm_inputs)

# ---------------------------------------------------------
# 5️⃣ Combine all outputs into one model
# ---------------------------------------------------------
combined_model = Model(
    inputs,
    [material_mean, material_std, build_class, setup_class],
    name="truss_costs_combined"
)

# ---------------------------------------------------------
# 6️⃣ Save the unified model
# ---------------------------------------------------------
combined_model.save("C:/models/truss_costs_combined")

print("✅ Combined model saved successfully to C:/models/truss_costs_combined")

INFO:tensorflow:Assets written to: C:/models/truss_costs_combined\assets
✅ Combined model saved successfully to C:/models/truss_costs_combined


In [10]:
import tensorflow as tf
import numpy as np

# ----------------------------------------------------------
# 1️⃣ Load your saved combined model
# ----------------------------------------------------------
model_path = "C:/models/truss_costs_combined"
model = tf.keras.models.load_model(model_path)

print("✅ Combined model loaded successfully!\n")

# ----------------------------------------------------------
# 2️⃣ Print model architecture and I/O shapes
# ----------------------------------------------------------
print("🧱 Model Summary:")
model.summary()

print("\n📥 Input shape:", model.input_shape)
print("📤 Output shapes:", [out.shape for out in model.outputs])
print("📤 Output names:", [out.name for out in model.outputs])

# ----------------------------------------------------------
# 3️⃣ Test a single input (one truss feature vector)
# ----------------------------------------------------------
# Example: 32-feature test vector in same order as training
x_test = np.array([[
    240.0, 4.0, 24.0, 3.9375, 0.0,
    42.0, 7.0, 0.0, 10.0, 90.0,
    24.389977, 960.0, 15.75, 0.01640625, 0.0,
    594.871, 40.0, 43.9375, 516705.0, 661.5,
    10080.0, 21600.0, 192.9375, 0.16666667,
    57600.0, 5.484797, 16.0, 15.503906,
    0.0, 0.0, 0.0, 0.0
]], dtype=np.float32)



print("\n🔍 Testing a single input of shape:", x_test.shape)

# ----------------------------------------------------------
# 4️⃣ Run inference
# ----------------------------------------------------------
outputs = model.predict(x_test)

# ----------------------------------------------------------
# 5️⃣ Print results
# ----------------------------------------------------------
print("\n✅ Model returned", len(outputs), "outputs:")
for i, out in enumerate(outputs):
    print(f"Output {i} ({model.output_names[i]}): shape={out.shape}, value={out}")


✅ Combined model loaded successfully!

🧱 Model Summary:
Model: "truss_costs_combined"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 truss_input (InputLayer)       [(None, 32)]         0           []                               
                                                                                                  
 normalization (Normalization)  (None, 32)           65          ['truss_input[0][0]']            
                                                                                                  
 ensemble_scaled (Functional)   [(None, 1),          947205      ['normalization[0][0]']          
                                 (None, 1)]                                                       
                                                                                                  
 build_cost_price_model

In [11]:
data = pd.read_csv("C:/models/data/truss_rebalanced.csv", quotechar='"')

# Remove targets to get features (32 columns)
X = data.drop(["material_cost_per_truss", "build_cost_per_truss", "setup_cost_per_run"], axis=1).values
combined_model = tf.keras.models.load_model("C:/models/truss_costs_combined")


metrics = evaluate_combined_model(combined_model, X, data, n_samples=5)


🔍 Sample Predictions vs Actuals:

Row 261535:
  Material → Pred:  68.18 ± 0.48 | Actual:  65.61
  Build    → Pred:  36.56 | Actual:  36.56
  Setup    → Pred:  49.86 | Actual:  49.86
------------------------------------------------------------
Row 34672:
  Material → Pred:  14.93 ± 0.10 | Actual:  13.98
  Build    → Pred:  13.85 | Actual:  13.85
  Setup    → Pred:  18.28 | Actual:  18.28
------------------------------------------------------------
Row 199993:
  Material → Pred:  44.05 ± 0.24 | Actual:  41.65
  Build    → Pred:  23.27 | Actual:  23.27
  Setup    → Pred:  45.43 | Actual:  45.43
------------------------------------------------------------
Row 97341:
  Material → Pred:  30.41 ± 0.07 | Actual:  31.15
  Build    → Pred:  27.70 | Actual:  27.70
  Setup    → Pred:  36.56 | Actual:  36.56
------------------------------------------------------------
Row 98319:
  Material → Pred:  35.91 ± 0.19 | Actual:  35.71
  Build    → Pred:  27.70 | Actual:  27.70
  Setup    → Pred:  36.56 |